In [1]:
from dotenv import dotenv_values
import os
from openai import OpenAI
import tiktoken


In [58]:
data=['For the people against it, let dogs rule you then','dogs promote active lifestyle']

In [59]:
proposal="dogs should be restricted from certain areas"

In [60]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

#### 

In [82]:
def convert_messages(contents):
    formatted_messages = []
    for content in contents:
        formatted_message = [
            {
                "role": "system",
                "content": f"""Classify user text based on support for the proposal: {proposal}. Use these guidelines:

Strongly Agree: Explicit, emphatic support for the proposal.
Agree: General support for the proposal. This includes interpreting sarcastic or rhetorical remarks as agreement if they imply that the alternative to the proposal is unreasonable or undesirable.
Neutral: Ambiguous or no clear position on the proposal.
Disagree: Opposes the proposal, directly or through positive mentions of opposing aspects.
Strongly Disagree: Explicit, emphatic opposition to the proposal.
When evaluating sarcasm or rhetorical language, focus on the implied stance rather than the literal wording. For example, a sarcastic remark suggesting an extreme alternative to the proposal likely indicates agreement with the proposal.

Output format:
Answer: """
            },
            {
                "role": "user",
                "content": (content.strip())
            }
        ]
        formatted_messages.append(formatted_message)
    return formatted_messages

In [83]:
messages=convert_messages(data)

In [ ]:


# Filter out comments longer than 3990 tokens
filtered_comments = [comment for comment in comments if len(comment) <= 3900]

# Now, filtered_comments will contain only the comments with 3900 characters or less


In [87]:
def estimate_cost_to_score_comments(model="gpt-3.5-turbo"):
    """Estimate the number of tokens in a dataframe's comment_body column"""
    num_tokens = 0
    completion=0
    filtered_messages=[]
    for message in messages:
        num = num_tokens_from_messages(message, model)
        if num <=3900:
            num_tokens+= num
            filtered_messages.append(message)
            completion +=5
    return ((num_tokens * (0.0023 / 1000))+(completion * (0.0031 / 1000) ),filtered_messages)

In [88]:
c,m=estimate_cost_to_score_comments()